# Beacon Chain model

Import necessary libraries

In [2]:
import pandas as pd
import math


Setting up global variables from Ethereum specification

In [3]:
BASE_REWARD_FACTOR=64*10**9
GWEI_DENOMINATOR=10**9
TIMELY_SOURCE_WEIGHT=14
TIMELY_TARGET_WEIGHT=26
TIMELY_HEAD_WEIGHT=14
SYNC_REWARD_WEIGHT=2
PROPOSER_WEIGHT=8
WEIGHT_DENOMINATOR=64
EPOCH_IN_DAY=225
SLOTS_IN_EPOCH=32
SYNC_COMMITTEE_SIZE=512
MIN_SLASHING_PENALTY_QUOTIENT_BELLATRIX=1/32
MAX_PROPOSER_SLASHINGS_PER_BLOCK=16
MAX_ATTESTER_SLASHINGS=2
PROPORTIONAL_SLASHING_MULTIPLIER=3
EPOCHS_PER_SLASHINGS_VECTOR=8192
INACTIVITY_PENALTY_QUOTIENT_BELLATRIX=2**24
INACTIVITY_SCORE_BIAS=4

Defining necessary functions

*All formulas are taken from Ethereum specification*

In [4]:
#Main modelling functions
def calculate_lido_attestation_rewards_penalties(base_reward,lido_avg_effective_balance,lido_down_vals,lido_vals,other_down_vals,other_avg_effective_balance,total_effective_balance):
    '''
    Calculate amount of attestation rewards and penalties within one oracle report.
    Return dictionary in the following format:

    {'rewards':X,
    'penalties':Y,
    'penalty':Z}

    Value of single penalty is needed for technical reasons and used in another functions'''
    assert lido_down_vals<=lido_vals, 'Amount of down vals higher than amount of vals'
    scaling_coef=1-(lido_down_vals*lido_avg_effective_balance+other_down_vals*other_avg_effective_balance)/total_effective_balance
    attestation_result={}
    increments=lido_avg_effective_balance//GWEI_DENOMINATOR
    lido_down_share=lido_down_vals/lido_vals
    attestation_result['rewards']=(TIMELY_SOURCE_WEIGHT+TIMELY_TARGET_WEIGHT+TIMELY_HEAD_WEIGHT)/WEIGHT_DENOMINATOR*increments*base_reward*(1-lido_down_share)*scaling_coef*lido_vals*EPOCH_IN_DAY
    attestation_result['penalties']=(TIMELY_SOURCE_WEIGHT+TIMELY_TARGET_WEIGHT)/WEIGHT_DENOMINATOR*increments*base_reward*lido_down_share*lido_vals*EPOCH_IN_DAY
    attestation_result['penalty']=(TIMELY_SOURCE_WEIGHT+TIMELY_TARGET_WEIGHT)/WEIGHT_DENOMINATOR*increments*base_reward
    return attestation_result

def calculate_proposer_reward(base_reward,total_vals,avg_effective_balance):
    '''
    Calculate single proposer reward both  for block proposal and for proposal due to sync committee contribution
    '''
    increments=avg_effective_balance//GWEI_DENOMINATOR
    reward_for_proposal=((total_vals/SLOTS_IN_EPOCH)*(PROPOSER_WEIGHT/(WEIGHT_DENOMINATOR-PROPOSER_WEIGHT))*(TIMELY_SOURCE_WEIGHT+TIMELY_TARGET_WEIGHT+TIMELY_HEAD_WEIGHT)/WEIGHT_DENOMINATOR*increments*base_reward)//1
    sync_committee_per_epoch=SYNC_REWARD_WEIGHT/(WEIGHT_DENOMINATOR*SLOTS_IN_EPOCH)*total_vals*increments*base_reward
    reward_for_sync_committee_proposal=sync_committee_per_epoch*(PROPOSER_WEIGHT/(WEIGHT_DENOMINATOR-PROPOSER_WEIGHT))//1
    return reward_for_proposal+reward_for_sync_committee_proposal

def calculate_sync_committee_rewards_penalties(base_reward,total_vals,avg_effective_balance,lido_vals,lido_down_vals):
    '''
    Calculate amount of rewards and penalties obtained due to sync committee participation.
    Return dictionary in the following format:

    {'rewards':X,
    'penalties':Y,
    'single_per_epoch':Z}

    Value of single reward for sync committee participation is needed for technical reasons and used in another functions
    '''
    assert lido_down_vals<=lido_vals, 'Amount of down vals higher than amount of vals'
    increments=avg_effective_balance//GWEI_DENOMINATOR
    sync_comittee_per_epoch=SYNC_REWARD_WEIGHT/WEIGHT_DENOMINATOR*total_vals*increments*base_reward
    lido_share=lido_vals/total_vals
    sync_result={}
    sync_result['rewards']=sync_comittee_per_epoch*EPOCH_IN_DAY*lido_share*(1-lido_down_vals/lido_vals)
    sync_result['penalties']=sync_comittee_per_epoch*EPOCH_IN_DAY*lido_share*lido_down_vals/lido_vals
    sync_result['single_per_epoch']=sync_comittee_per_epoch/SYNC_COMMITTEE_SIZE
    return sync_result


def calculate_rewards_for_proposal(proposer_reward,lido_vals,total_vals,lido_down_vals):
    '''
    Knowing current reward for proposal calculates how many GWEI will be earned as a proposal rewards within one oracle report.
    Amount of rewards is taken as Lido share among all validators multiplied by all proposal rewards for one oracle report period multiplied by
    share of Lido active validators
    '''
    total_proposer_rewards=proposer_reward*SLOTS_IN_EPOCH*EPOCH_IN_DAY
    lido_share=lido_vals/total_vals
    down_coef=1-lido_down_vals/lido_vals
    return total_proposer_rewards*lido_share*down_coef

def calculate_el_rewards(median_el_reward,lido_vals,lido_down_vals,total_vals):
   '''
   Knowing current median EL reward per block  calculates how many GWEI will be earned as a EL rewards within one oracle report.
   Amount of rewards is taken as Lido share among all validators multiplied by all EL rewards for one oracle report period multiplied by
   share of Lido active validators
   As a default we assume that EL rewards equal 0
   '''

   total_el_reward=median_el_reward*SLOTS_IN_EPOCH*EPOCH_IN_DAY
   lido_share=lido_vals/total_vals
   down_coef=1-lido_down_vals/lido_vals
   return total_el_reward*lido_share*down_coef

def calculate_initial_slashing(lido_avg_balance,new_lido_slashings):
    '''
    Calculate how many GWEI would be loosed due to initial slashing penalties which happened within oracle report
    '''
    assert new_lido_slashings<=(MAX_PROPOSER_SLASHINGS_PER_BLOCK+MAX_ATTESTER_SLASHINGS)*SLOTS_IN_EPOCH*EPOCH_IN_DAY,'Amount of slashed validators cannot be processed within 1 day'
    return lido_avg_balance*MIN_SLASHING_PENALTY_QUOTIENT_BELLATRIX*new_lido_slashings

def calculate_correlation_slashing(midterm_lido_slashings,network_effective_balance,total_slashed_vals,network_avg_effective_balance,lido_avg_effective_balance):
    '''
    Knowing amount of ongoing slashing in the network and amount of Lido validators which would receive midterm slashing penalty calculates how many GWEi would be lost because of midterm slashing
    '''
    assert midterm_lido_slashings<=total_slashed_vals, 'Midterm slashed amount higher than total slashed amount'
    lido_avg_effective_balance-=lido_avg_effective_balance*MIN_SLASHING_PENALTY_QUOTIENT_BELLATRIX
    slashed_increments=total_slashed_vals*(network_avg_effective_balance//GWEI_DENOMINATOR)
    correlation_slashing=min(lido_avg_effective_balance,PROPORTIONAL_SLASHING_MULTIPLIER*lido_avg_effective_balance*slashed_increments/(network_effective_balance//GWEI_DENOMINATOR))
    return midterm_lido_slashings*correlation_slashing

def calculate_inactivity_leak_penalty(lido_avg_balance,other_avg_balance,epoch_in_inactivity_leak):
    '''
    Calculate value of inactivity leak penalty both for Lido and other validators for one epoch
    Return result as a dictionary:

    {'penalty_lido':X,
    'penalty_others:Y}

    '''
    score=INACTIVITY_SCORE_BIAS*epoch_in_inactivity_leak
    inactivity_penalty={}
    inactivity_penalty['penalty_lido']=(score*lido_avg_balance)/(INACTIVITY_SCORE_BIAS*INACTIVITY_PENALTY_QUOTIENT_BELLATRIX)//1
    inactivity_penalty['penalty_others']=(score*other_avg_balance)/(INACTIVITY_SCORE_BIAS*INACTIVITY_PENALTY_QUOTIENT_BELLATRIX)//1
    return inactivity_penalty

def calculate_inactivity_leak(base_reward,sync_committee_penalty,total_vals,lido_down_vals,other_down_vals,network_effective_balance,lido_avg_real_balance,other_avg_real_balance,lido_avg_effective_balance,other_avg_effective_balance,epoch_in_inactivity):
    '''
    Simulate impact of inactivity leak, unlike other functions calculate result for each epoch. Return dictionary which could be devided into 2 logical parts:
    Oracle data - info needed for oracle report (total_penalty and total_epochs)
    Technical data - info needed for calculation of inactivity leak impact in future reports
    '''

    epoch_in_inactivity+=1
    total_down_balance=lido_down_vals*lido_avg_effective_balance+other_down_vals*other_avg_effective_balance
    total_active_balance=network_effective_balance-total_down_balance
    amount_of_sync_commettee_vals=get_probability_outcomes(lido_down_vals, total_vals)#get most probable amount of lido down validators which would be assigned to sync committee
    inactivity_leak_result={}  #we calculate impact only for one oracle epochs, info about previous inactivity leak impact is passed as epoch_in_inactivity parameter
    inactivity_leak_result['attestation_penalty']=0
    inactivity_leak_result['inactivity_penalty']=0
    inactivity_leak_result['total_penalty']=inactivity_leak_result['attestation_penalty']+inactivity_leak_result['inactivity_penalty']


    while total_active_balance/network_effective_balance <0.6666666 and  epoch_in_inactivity%EPOCH_IN_DAY!=0:
        #adjust effective balance to real balance
        if lido_avg_real_balance<lido_avg_effective_balance:
            lido_avg_effective_balance=(lido_avg_real_balance+0.25*GWEI_DENOMINATOR)//GWEI_DENOMINATOR*GWEI_DENOMINATOR
        if other_avg_real_balance<other_avg_effective_balance:
            other_avg_effective_balance=(other_avg_real_balance+0.25*GWEI_DENOMINATOR)//GWEI_DENOMINATOR*GWEI_DENOMINATOR

        #Calculate impact of imactivity leak
        inactivity_penalty=calculate_inactivity_leak_penalty(lido_avg_balance=lido_avg_effective_balance,other_avg_balance=other_avg_effective_balance,epoch_in_inactivity_leak=epoch_in_inactivity)
        lido_increments=(lido_avg_effective_balance+0.25*GWEI_DENOMINATOR)//GWEI_DENOMINATOR
        other_increments=(other_avg_effective_balance+0.25*GWEI_DENOMINATOR)//GWEI_DENOMINATOR
        lido_attestation_penalty=(TIMELY_SOURCE_WEIGHT+TIMELY_TARGET_WEIGHT)/WEIGHT_DENOMINATOR*lido_increments*base_reward
        other_attestation_penalty=(TIMELY_SOURCE_WEIGHT+TIMELY_TARGET_WEIGHT)/WEIGHT_DENOMINATOR*other_increments*base_reward
        inactivity_leak_result['inactivity_penalty'] +=inactivity_penalty['penalty_lido']*lido_down_vals/GWEI_DENOMINATOR
        inactivity_leak_result['attestation_penalty'] +=(lido_attestation_penalty*lido_down_vals+ sync_committee_penalty*amount_of_sync_commettee_vals)/GWEI_DENOMINATOR
        lido_avg_real_balance -=inactivity_penalty['penalty_lido']+lido_attestation_penalty
        other_avg_real_balance -=inactivity_penalty['penalty_others']+other_attestation_penalty
        total_decrease=lido_attestation_penalty*lido_down_vals+other_attestation_penalty*other_down_vals+inactivity_penalty['penalty_lido']*lido_down_vals+inactivity_penalty['penalty_others']*other_down_vals
        network_effective_balance -=total_decrease
        epoch_in_inactivity+=1

    #gather all info into final dictionary
    inactivity_leak_result['total_penalty']=inactivity_leak_result['attestation_penalty']+inactivity_leak_result['inactivity_penalty']
    inactivity_leak_result['total_epochs']=epoch_in_inactivity
    inactivity_leak_result['epoch_this_day']=inactivity_leak_result['total_epochs']-EPOCH_IN_DAY
    if inactivity_leak_result['epoch_this_day'] <=0:
        inactivity_leak_result['epoch_this_day']=EPOCH_IN_DAY
    inactivity_leak_result['network_effective_balance']=network_effective_balance/GWEI_DENOMINATOR
    inactivity_leak_result['lido_avg_real_balance']=lido_avg_real_balance
    inactivity_leak_result['other_avg_real_balance']=other_avg_real_balance
    inactivity_leak_result['lido_avg_effective_balance']=lido_avg_effective_balance
    inactivity_leak_result['other_avg_effective_balance']=other_avg_effective_balance
    return inactivity_leak_result



def calculate_oracle_report_flow(lido_down_vals,other_down_vals,lido_new_slashing,lido_ongoing_slashing,lido_midterm_slashing,other_ongoing_slashing,network_state,epoch_in_inactivity):
    '''
    Main function of model. Gather all variables which describe network state and events which happened in Beacon Chain and with other functions generate report for oracle.
    '''
    #in this block we obtain value of variables which is needed for final calculations
    total_vals=get_total_amount_vals(lido_vals=network_state['lido_vals'],other_vals=network_state['other_vals'])
    network_effective_balance=calculate_network_effective_balance(lido_avg_effective_balance=network_state['lido_avg_effective_balance'],lido_vals=network_state['lido_vals'],other_avg_effective_balance=network_state['other_avg_effective_balance'],other_vals=network_state['other_vals'])
    network_avg_effective_balance=calculate_network_avg_effective_balance(total_vals=total_vals,total_balance=network_effective_balance)
    base_reward=calculate_base_reward(network_effective_balance)
    proposer_reward=calculate_proposer_reward(base_reward=base_reward,total_vals=total_vals,avg_effective_balance=network_avg_effective_balance)
    total_lido_down_vals=lido_down_vals+lido_new_slashing+lido_ongoing_slashing+lido_midterm_slashing
    total_other_down_vals=other_down_vals+other_ongoing_slashing
    total_down_balance= total_lido_down_vals*network_state['lido_avg_effective_balance']+other_down_vals*network_state['other_avg_effective_balance']
    total_slashed_vals=lido_new_slashing+lido_ongoing_slashing+lido_midterm_slashing+other_ongoing_slashing
    total_active_balance=network_effective_balance-total_down_balance

    #by default inactivity leak mode is disabled
    inactivity_leak=False


    #final calculations
    attest_result=calculate_lido_attestation_rewards_penalties(base_reward=base_reward,lido_avg_effective_balance=network_state['lido_avg_effective_balance'],lido_down_vals=total_lido_down_vals,lido_vals=network_state['lido_vals'],other_down_vals=total_other_down_vals,other_avg_effective_balance=network_state['other_avg_effective_balance'],total_effective_balance=network_effective_balance)
    proposal_rewards=calculate_rewards_for_proposal(proposer_reward=proposer_reward,lido_vals=network_state['lido_vals'],total_vals=total_vals,lido_down_vals=total_lido_down_vals)
    sync_committee_result=calculate_sync_committee_rewards_penalties(base_reward=base_reward,total_vals=total_vals,avg_effective_balance=network_avg_effective_balance,lido_vals=network_state['lido_vals'],lido_down_vals=total_lido_down_vals)
    el_rewards=calculate_el_rewards(median_el_reward=network_state['median_EL_reward'],lido_vals=network_state['lido_vals'],lido_down_vals=total_lido_down_vals,total_vals=total_vals)
    initial_penalties=calculate_initial_slashing(lido_avg_balance=network_state['lido_avg_effective_balance'],new_lido_slashings=lido_new_slashing)
    correlation_slashings=calculate_correlation_slashing(midterm_lido_slashings=lido_midterm_slashing,network_effective_balance=network_effective_balance,total_slashed_vals= total_slashed_vals,network_avg_effective_balance= network_avg_effective_balance,lido_avg_effective_balance=network_state['lido_avg_effective_balance'])

    #If more than 1/3 of network is down Beacon Chain enters inactivity leak mode
    if total_active_balance/network_effective_balance<0.6666666:
        inactivity_leak=True
        inactivity_leak_result=calculate_inactivity_leak(base_reward=base_reward,sync_committee_penalty=sync_committee_result['single_per_epoch'],total_vals=total_vals,lido_down_vals=total_lido_down_vals,other_down_vals=other_down_vals,network_effective_balance=network_effective_balance,lido_avg_real_balance=network_state['lido_avg_real_balance'],other_avg_real_balance=network_state['other_avg_real_balance'], lido_avg_effective_balance=network_state['lido_avg_effective_balance'],other_avg_effective_balance=network_state['lido_avg_effective_balance'],epoch_in_inactivity=epoch_in_inactivity)



    #create oracle report
    oracle_report={}

    oracle_report['attestation_rewards']=attest_result['rewards']/GWEI_DENOMINATOR
    oracle_report['attestation_penalties']=attest_result['penalties']/GWEI_DENOMINATOR
    oracle_report['proposal_reward']=proposal_rewards/GWEI_DENOMINATOR
    oracle_report['sync_committee_reward']=sync_committee_result['rewards']/GWEI_DENOMINATOR
    oracle_report['sync_committee_penalty']=sync_committee_result['penalties']/GWEI_DENOMINATOR
    oracle_report['initial_slashing_penalties']=initial_penalties/GWEI_DENOMINATOR
    oracle_report['correlation_slashing_penalties']=correlation_slashings/GWEI_DENOMINATOR
    oracle_report['total_att_loss_for_new_slashing']=attest_result['penalty']*lido_new_slashing*EPOCHS_PER_SLASHINGS_VECTOR/GWEI_DENOMINATOR
    oracle_report['inactivity_leak_penalty']=0
    oracle_report['epoch_in_inactivity']=0
    oracle_report['el_rewards']=el_rewards/GWEI_DENOMINATOR
    if inactivity_leak:
        oracle_report['attestation_rewards']=0
        oracle_report['attestation_penalties']=inactivity_leak_result['attestation_penalty']
        oracle_report['inactivity_leak_penalty']=inactivity_leak_result['inactivity_penalty']
        oracle_report['epoch_in_inactivity']=inactivity_leak_result['total_epochs']
        oracle_report['el_rewards']=0
    oracle_report['cl_rewards']=oracle_report['attestation_rewards']+oracle_report['proposal_reward']+oracle_report['sync_committee_reward']
    oracle_report['total_rewards']=oracle_report['cl_rewards']+oracle_report['el_rewards']
    oracle_report['total_penalties']=oracle_report['attestation_penalties']+oracle_report['sync_committee_penalty']+oracle_report['initial_slashing_penalties']+oracle_report['correlation_slashing_penalties']+oracle_report['inactivity_leak_penalty']
    oracle_report['total_delta']=oracle_report['total_rewards']-oracle_report['total_penalties']

    return oracle_report

#Wrappers for different scenarios modelling
def get_edge_slashing(network_state):
    '''
    With given network state calculate how many lido validators should be slashed within one oracle report in order to receive negative rebase
    '''
    delta=1
    slashed=1
    while delta>0 and slashed<200000:
        result=calculate_oracle_report_flow(lido_down_vals=0,other_down_vals=0,lido_new_slashing=slashed,lido_ongoing_slashing=0,lido_midterm_slashing=0,other_ongoing_slashing=0,network_state=network_state,epoch_in_inactivity=0)
        delta=result['total_delta']
        slashed+=1
    return slashed

def get_edge_down(network_state):
    '''
    With given network state calculate how many lido validators should be inactive within one oracle report  in order to receive negative rebase
    '''
    delta=1
    down=1
    while delta>0 and down<200000:
        result=calculate_oracle_report_flow(lido_down_vals=down,other_down_vals=0,lido_new_slashing=0,lido_ongoing_slashing=0,lido_midterm_slashing=0,other_ongoing_slashing=0,network_state=network_state,epoch_in_inactivity=0)
        delta=result['total_delta']
        down+=1
    return down


def get_edge_midterm(network_state):
    '''
    With given network state calculates how many Lido validators should receive midterm slashing penalty in order to receive negative rebase.
    Return result as a dictionary with minimum Lido validators (and accordingly maximum other validators) and vice versa (maximum Lido and minimum others) in order to receive negative rebase
    '''
    cases=[]
    result=[]
    max_slashed=get_edge_slashing(network_state)

    for lido_slashed in range(0,max_slashed,1):
        for lido_midterm_slashed in range(0,200000,100):
                delta=calculate_oracle_report_flow(lido_down_vals=0,other_down_vals=0,lido_new_slashing=0,lido_ongoing_slashing=0,lido_midterm_slashing=lido_slashed,other_ongoing_slashing=lido_midterm_slashed,network_state=network_state,epoch_in_inactivity=0)
                if delta['total_delta']<0:
                    row={}
                    row['lido_midterm_slashed']=lido_slashed
                    row['other_slashed']=lido_midterm_slashed
                    cases.append(row)
                    break


    midterm_slashed=pd.DataFrame(cases)
    lido_min={'lido':midterm_slashed[midterm_slashed['lido_midterm_slashed']==midterm_slashed['lido_midterm_slashed'].min()].iloc[0,0],
              'other':midterm_slashed[midterm_slashed['lido_midterm_slashed']==midterm_slashed['lido_midterm_slashed'].min()].iloc[0,1]}
    lido_max={'lido':midterm_slashed[midterm_slashed['other_slashed'] == midterm_slashed['other_slashed'].min()].sort_values(
    by='lido_midterm_slashed').iloc[0,0],
              'other':midterm_slashed[midterm_slashed['other_slashed'] == midterm_slashed['other_slashed'].min()].sort_values(
    by='lido_midterm_slashed').iloc[0,1]}

    result.append(lido_min)
    result.append(lido_max)
    return result

def get_midterm_freeze_other(network_scenario,other_slashed):
    '''
    Needed for specific scenarios calculation. Answer the question 'With given amount of ongoing slashings, how many Lido validators should receive midterm slashing penalty
    in order to receive negative rebase?'
     '''
    delta=1
    slashed=1
    while delta>0 and slashed<200000:
        result=calculate_oracle_report_flow(lido_down_vals=0,other_down_vals=0,lido_new_slashing=0,lido_ongoing_slashing=0,lido_midterm_slashing=slashed,other_ongoing_slashing=other_slashed,network_state=network_scenario,epoch_in_inactivity=0)
        delta=result['total_delta']
        slashed+=1
    return slashed


def get_midterm_freeze_lido(network_scenario,lido_slashed):
    '''
    Needed for specific scenarios calculation. Answer the question 'With given amount of Lido validators which receive midterm slashing penalty how many ongoing slashings should be in beacon Chain  in order to receive negative rebase?'
    '''
    delta=1
    slashed=1
    while delta>0 and slashed<200000:
        result=calculate_oracle_report_flow(lido_down_vals=0,other_down_vals=0,lido_new_slashing=0,lido_ongoing_slashing=0,lido_midterm_slashing=lido_slashed,other_ongoing_slashing=slashed,network_state=network_scenario,epoch_in_inactivity=0)
        delta=result['total_delta']
        slashed+=1
    return slashed


def simulate_scenarios(network_scenarios):
    '''
    Find edge cases (amount of slashings, lido down validators, amount of midterm slashings) for any specified network scenario
    '''
    result={}
    for x in range(len(network_scenarios)):
        single_result={}
        single_result['slashed']=get_edge_slashing(network_scenarios[x])
        single_result['lido_down']=get_edge_down(network_scenarios[x])
        midterm=get_edge_midterm(network_scenarios[x])
        single_result['non_Lido_slashing_Lido_min']=midterm[0]
        single_result['non_Lido_slashing_Lido_max']=midterm[1]
        result[network_scenarios[x]['name']]=single_result

    return result

def simulate_scenarios_with_freeze(network_scenario,lido_slashed,other_slashed):
    '''
    Find edge cases (amount of slashings, lido down validators, amount of midterm slashings) for all specified network scenario  and amount of lido slashed validators/ ongoing slashings
    '''
    result={}
    result['slashed']=get_edge_slashing(network_scenario)
    result['lido_down']=get_edge_down(network_scenario)
    result['non_Lido_slashing_Lido_min']=get_midterm_freeze_other(network_scenario,other_slashed)
    result['non_Lido_slashing_Lido_max']=get_midterm_freeze_lido(network_scenario,lido_slashed)


    return result

def get_scenarios_EL_change(network_scenarios,el_reward):
  '''
  Find edge cases for all specified scenarios with custom EL reward
  '''
  scenarios_to_change=[]
  for x in range(len(network_scenarios)):
    scenarios_to_change.append(network_scenarios[x].copy())
    scenarios_to_change[x]['median_EL_reward']= el_reward

  result=simulate_scenarios(scenarios_to_change)
  return result

def get_scenarios_EL_change_with_freeze(network_scenario,el_reward,lido_slashed,other_slashed):
  '''
  Find edge cases for all specified scenarios with custom EL reward and amount of lido slashed validators/ ongoing slashings
  '''
  scenario_to_change=network_scenario.copy()
  scenario_to_change['median_EL_reward']=el_reward

  result=simulate_scenarios_with_freeze(scenario_to_change,lido_slashed,other_slashed)
  return result

#Helpers
def get_total_amount_vals(lido_vals,other_vals):
    return lido_vals+other_vals

def calculate_network_effective_balance(lido_avg_effective_balance,lido_vals,other_avg_effective_balance,other_vals):
    return lido_avg_effective_balance*lido_vals+other_avg_effective_balance*other_vals

def calculate_network_avg_effective_balance(total_vals,total_balance):
    return round((total_balance//GWEI_DENOMINATOR)/total_vals)*GWEI_DENOMINATOR

def calculate_base_reward(network_effective_balance):
    return (BASE_REWARD_FACTOR/math.sqrt(network_effective_balance))//1

def total_slashed_vals(new_lido_slashings,midterm_lido_slashings,ongoing_lido_slashings,ongoing_other_slashing):
    return new_lido_slashings+midterm_lido_slashings+ongoing_lido_slashings+ongoing_other_slashing

def c(n, k):   # helper for large numbers binomial coefficient calculation
    if 0 <= k <= n:
        nn = 1
        kk = 1
        for t in range(1, min(k, n - k) + 1):

            nn *= n
            kk *= t
            n -= 1
        return nn // kk
    else:
        return 0

def get_probability_outcomes(down_vals,total_vals):
    outcome = []
    for offline_validator_sync_cnt in range(1, SYNC_COMMITTEE_SIZE+1):
        outcome.append(c(int(down_vals), offline_validator_sync_cnt)*c(int(total_vals-down_vals),SYNC_COMMITTEE_SIZE-offline_validator_sync_cnt)/c(int(total_vals),SYNC_COMMITTEE_SIZE))

    df_outcome = pd.DataFrame(pd.Series(outcome), columns=['outcome'])

    return df_outcome.index[df_outcome['outcome'].idxmax()]

Predicted network state, you can add you own state in following format:

{'name': scenario name,
'lido_vals':amount of Lido validators,
'other_vals':amount of Other validators,
'lido_avg_effective_balance':Lido validator average effective balance,
'other_avg_effective_balance':Other validators average effective balance,
'lido_avg_real_balance':Lido validator average real balance,
'other_avg_real_balance':Other validator average real balance,
'median_EL_reward': median EL reward per block
}

In [7]:
network_scenarios=[{
    'name':'expected',
    'lido_vals':149950,
    'other_vals':373639,
    'lido_avg_effective_balance':32*10**9,
    'other_avg_effective_balance':32*10**9,
    'lido_avg_real_balance':32*10**9,
    'other_avg_real_balance':32*10**9,
    'median_EL_reward': 0
    },
    {
    'name':'extreme_growth',
    'lido_vals':148435,
    'other_vals':516900,
    'lido_avg_effective_balance':32*10**9,
    'other_avg_effective_balance':32*10**9,
    'lido_avg_real_balance':32*10**9,
    'other_avg_real_balance':32*10**9,
    'median_EL_reward': 0
    },
    {
    'name':'extreme_stagnation',
    'lido_vals':144858,
    'other_vals':350152,
    'lido_avg_effective_balance':32*10**9,
    'other_avg_effective_balance':32*10**9,
    'lido_avg_real_balance':32*10**9,
    'other_avg_real_balance':32*10**9,
    'median_EL_reward': 0
    },
    {
    'name':'extreme_post_shanghai',
    'lido_vals':100983,
    'other_vals':205294,
    'lido_avg_effective_balance':32*10**9,
    'other_avg_effective_balance':32*10**9,
    'lido_avg_real_balance':32*10**9,
    'other_avg_real_balance':32*10**9,
    'median_EL_reward': 0
    }]


## Usage examples

*Single state simulation*

In [9]:
calculate_oracle_report_flow(lido_down_vals=0,other_down_vals=0,lido_new_slashing=200,lido_ongoing_slashing=0,lido_midterm_slashing=0,other_ongoing_slashing=0,network_state=network_scenarios[0],epoch_in_inactivity=0)

{'attestation_rewards': 449.2355733750852,
 'attestation_penalties': 0.44459999999999994,
 'proposal_reward': 66.57884845556343,
 'sync_committee_reward': 16.6447125,
 'sync_committee_penalty': 0.022230000000000003,
 'initial_slashing_penalties': 200.0,
 'correlation_slashing_penalties': 0.0,
 'total_att_loss_for_new_slashing': 16.187392,
 'inactivity_leak_penalty': 0,
 'epoch_in_inactivity': 0,
 'el_rewards': 0.0,
 'cl_rewards': 532.4591343306486,
 'total_rewards': 532.4591343306486,
 'total_penalties': 200.46683,
 'total_delta': 331.9923043306486}

*Find edge cases for all scenarios*

In [14]:
pd.DataFrame(simulate_scenarios(network_scenarios))

,expected,extreme_growth,extreme_stagnation,extreme_post_shanghai
slashed,531,467,528,467
lido_down,85169,85503,82162,56798
lido_midterm_min,"{'lido': 12, 'other': 187700}","{'lido': 13, 'other': 194200}","{'lido': 12, 'other': 174700}","{'lido': 7, 'other': 195300}"
lido_midterm_max,"{'lido': 526, 'other': 5100}","{'lido': 462, 'other': 6700}","{'lido': 522, 'other': 4800}","{'lido': 466, 'other': 2800}"


*Find edge cases custom EL reward*

In [15]:
pd.DataFrame(get_scenarios_EL_change(network_scenarios,0.0234*10**9))

,expected,extreme_growth,extreme_stagnation,extreme_post_shanghai
slashed,579,504,576,522
lido_down,88600,88468,85573,59816
lido_midterm_min,"{'lido': 14, 'other': 173800}","{'lido': 14, 'other': 198300}","{'lido': 13, 'other': 195000}","{'lido': 9, 'other': 190300}"
lido_midterm_max,"{'lido': 570, 'other': 5100}","{'lido': 503, 'other': 6600}","{'lido': 567, 'other': 4800}","{'lido': 514, 'other': 2800}"


*Find edge cases for scenario with given amount of Lido validators, which receive midterm slashing penalty, and network ongoing slashings*

In [16]:
simulate_scenarios_with_freeze(network_scenario=network_scenarios[0],lido_slashed=660,other_slashed=4000)

{'slashed': 531,
 'lido_down': 85169,
 'lido_midterm_freeze_other': 640,
 'lido_midterm_freeze_lido': 3825}

*Find edge cases for scenario with given amount of Lido validators, which receive midterm slashing penalty, and network ongoing slashings and custom EL reward*

In [17]:
get_scenarios_EL_change_with_freeze(network_scenario=network_scenarios[0],el_reward=0.0234*10**9,lido_slashed=660,other_slashed=4000)

{'slashed': 579,
 'lido_down': 88600,
 'lido_midterm_freeze_other': 690,
 'lido_midterm_freeze_lido': 4232}

### For publication

This section contains code and tables for publication

In [5]:
inactivity_leak_result={
  'expected': {'slashed': 81,
  'lido_down': 1637,
  'non_Lido_slashing_Lido_min': {'lido': 3, 'other': 156500},
  'non_Lido_slashing_Lido_max': {'lido': 80, 'other': 5500}},
 'extreme_growth': {'slashed': 70,
  'lido_down': 1451,
  'non_Lido_slashing_Lido_min': {'lido': 3, 'other': 175000},
  'non_Lido_slashing_Lido_max': {'lido': 69, 'other': 7180}},
 'extreme_stagnation': {'slashed': 79,
  'lido_down': 1625,
  'non_Lido_slashing_Lido_min': {'lido': 3, 'other': 149900},
  'non_Lido_slashing_Lido_max': {'lido': 78, 'other': 5300}},
 'extreme_post_shanghai': {'slashed': 70,
  'lido_down': 1415,
  'non_Lido_slashing_Lido_min': {'lido': 3, 'other': 8100},
  'non_Lido_slashing_Lido_max': {'lido': 69, 'other': 3300}}}

combination_result={
    'expected':{
        'lido_down':29564,
        'others_down':30000,
        'lido_new_slashing':124,
        'lido_ongoing_slashing':196,
        'lido_midterm_slashing':196,
        'other_ongoing_slashing':4871
    },
    'extreme_growth':{
        'lido_down':29564,
        'others_down':30000,
        'lido_new_slashing':132,
        'lido_ongoing_slashing':186,
        'lido_midterm_slashing':201,
        'other_ongoing_slashing':4759
    },
    'extreme_stagnation':{
        'lido_down':28564,
        'others_down':25000,
        'lido_new_slashing':132,
        'lido_ongoing_slashing':175,
        'lido_midterm_slashing':211,
        'other_ongoing_slashing':4017
    },
    'extreme_post_shanghai':{
        'lido_down':22173,
        'others_down':25000,
        'lido_new_slashing':98,
        'lido_ongoing_slashing':117,
        'lido_midterm_slashing':121,
        'other_ongoing_slashing':3661
    }
}

In [8]:
pd.DataFrame(simulate_scenarios(network_scenarios)).style.set_caption("Output: individual params (w/o inactivity leak)")

,expected,extreme_growth,extreme_stagnation,extreme_post_shanghai
slashed,531,467,528,467
lido_down,85169,85503,82162,56798
non_Lido_slashing_Lido_min,"{'lido': 12, 'other': 187700}","{'lido': 13, 'other': 194200}","{'lido': 12, 'other': 174700}","{'lido': 7, 'other': 195300}"
non_Lido_slashing_Lido_max,"{'lido': 526, 'other': 5100}","{'lido': 462, 'other': 6700}","{'lido': 522, 'other': 4800}","{'lido': 466, 'other': 2800}"


In [9]:
pd.DataFrame(inactivity_leak_result).style.set_caption("Output: individual params (with inactivity leak)")

,expected,extreme_growth,extreme_stagnation,extreme_post_shanghai
slashed,81,70,79,70
lido_down,1637,1451,1625,1415
non_Lido_slashing_Lido_min,"{'lido': 3, 'other': 156500}","{'lido': 3, 'other': 175000}","{'lido': 3, 'other': 149900}","{'lido': 3, 'other': 8100}"
non_Lido_slashing_Lido_max,"{'lido': 80, 'other': 5500}","{'lido': 69, 'other': 7180}","{'lido': 78, 'other': 5300}","{'lido': 69, 'other': 3300}"


In [10]:
pd.DataFrame(combination_result).style.set_caption("Output: params combination (w/o inactivity leak)")

,expected,extreme_growth,extreme_stagnation,extreme_post_shanghai
lido_down,29564,29564,28564,22173
others_down,30000,30000,25000,25000
lido_new_slashing,124,132,132,98
lido_ongoing_slashing,196,186,175,117
lido_midterm_slashing,196,201,211,121
other_ongoing_slashing,4871,4759,4017,3661
